##  CASE STUDY 1 - Uber Drive Analysis

Steps - 
1.Import the libraries

2.Get the data and observe it

3.Check missing values, either remove it or fill it.

4.Get summary of data using python function.

5.Explore the data parameter wise

Here we have information of destination(start and stop), time(start and stop), category and purpose of trip, miles covered.


In [ ]:
# Import the libraries 
import numpy as np
import pandas as pd
#

In [ ]:
# Automatically print output of each line of code
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [ ]:
# Get the Data 
df = pd.read_csv('uberdrive.csv')

#View the first 5 rows of data
df.head()

In [ ]:
#View the last 5 rows of data
df.tail()

In [ ]:
# understand shape and size of data 
print("Shape of uber df: ",df.shape, "Size of uber df: ", df.size, '\n')

**The dataset has 1156 rows and 7 columns**

In [ ]:
#Get a summary of the numerical columns in the data
df.describe(include='all')

**The miles driven ranges from 0.5 miles to 12204 miles with an average of 21 miles**

In [ ]:
#get more information about data
df.info()

**The dataset has 1 numerical variable and 6 categorical variables** <br>
**The "PURPOSE" column has many missing values**

In [ ]:
#Get the number of missing values in each column
df.isnull().sum()

In [ ]:
# Get the initial data with dropping the NA values
df = df.dropna()

#Get the shape of the dataframe after removing the null values
df.shape

**The dataset now contains 653 rows of non-null values**

In [ ]:
#get the summary of data
df.describe().T

### Lets explore the data parameter wise - 

1.Destination - (starting and stopping)

2.Time - (hour of the day, day of week, month of year)

3.Categories

4.Purpose 

5.Grouping two parameters to get more insights


# 1.Destination - (starting and stopping)

In [ ]:
# Get the starting destination, unique destination
print(df['START*'].unique()) #names of unique start points
print(len(df['START*'].unique())) #count of unique start points

**There are 131 unique start destinations in the dataset**

In [ ]:
# Get the starting destination, unique destination
print(df['STOP*'].unique()) #names of unique start points
print(len(df['STOP*'].unique())) #count of unique start points
df['STOP*'].nunique() # Another way to get unique Stop location count

**There are 137 unique start destinations in the dataset**

In [ ]:
#Identify popular start destinations - top 10
df['START*'].value_counts().head(10)

**We can say that Cary is most poplular starting point for this driver.**

In [ ]:
#Identify popular stop destinations - top 10
df['STOP*'].value_counts().head(10)

**Cary also features in the most popular stop destinations**

In [ ]:
#Find out most farthest start and stop pair -top10
#Dropping Unknown Location Value
df2 = df[df['START*']!= 'Unknown Location']
df2 = df2[df2['STOP*']!= 'Unknown Location']
df2.groupby(['START*','STOP*'])['MILES*'].sum().sort_values(ascending=False).head(10)



**Latta and Jacksonville are the farthest from each other**

In [ ]:
#Find out most popular start and stop pair - top10
df2.groupby(['START*','STOP*']).size().sort_values(ascending=False).head(10)


**The most popular start to destination pair is Cary-Morrisville**

## 2. Manipulating date & time objects

In [ ]:
# START DATE and END_DATE have string format. Convert it to datetime object

df.loc[:, 'START_DATE*'] = df['START_DATE*'].apply(lambda x: pd.datetime.strptime(x, '%m/%d/%Y %H:%M'))
df.loc[:, 'END_DATE*'] = df['END_DATE*'].apply(lambda x: pd.datetime.strptime(x, '%m/%d/%Y %H:%M'))

In [ ]:
df.info()
df.head(10)

In [ ]:
#Calculate the duration for the rides
df['DIFF'] = df['END_DATE*'] - df['START_DATE*']
print(df['DIFF'])

In [ ]:
#convert duration to numbers(minutes)
df.loc[:, 'DIFF'] = df['DIFF'].apply(lambda x: pd.Timedelta.to_pytimedelta(x).days/(24*60) + pd.Timedelta.to_pytimedelta(x).seconds/60)

In [ ]:
df['DIFF'].head()

In [ ]:
df['DIFF'].describe()

**Ride durations range from 2 minutes to 330 minutes with an average duration of 23 minutes**

In [ ]:
#Capture Hour, Day, Month and Year of Ride in a separate column
df['month'] = pd.to_datetime(df['START_DATE*']).dt.month
df['Year'] = pd.to_datetime(df['START_DATE*']).dt.year
df['Day'] = pd.to_datetime(df['START_DATE*']).dt.day
df['Hour'] = pd.to_datetime(df['START_DATE*']).dt.hour

In [ ]:
#Capture day of week and rename to weekday names
df['day_of_week'] = pd.to_datetime(df['START_DATE*']).dt.dayofweek
days = {0:'Mon',1:'Tue',2:'Wed',3:'Thur',4:'Fri',5:'Sat',6:'Sun'}

df['day_of_week'] = df['day_of_week'].apply(lambda x: days[x])

In [ ]:
#Rename the numbers in the Month column to calendar months
import calendar
df['month'] = df['month'].apply(lambda x: calendar.month_abbr[x])
df.head()

In [ ]:
#Extract the total number of trips per month, weekday
print(df['month'].value_counts())
print(df['day_of_week'].value_counts())

**December has maximum number of trips and August has the least** <br>
**Friday has the maximum number of trips**

In [ ]:
#Getting the average distance covered per month
df.groupby('month').mean()['MILES*'].sort_values(ascending = False)


**Longest average distance is covered in Oct and least in Dec**

In [ ]:
#Number of trips based of hour of day
df['Hour'].value_counts()

**Afternoons and evenings seem to have the maximum number of trips**

In [ ]:
# calculate trip speed for each trip
df['Duration_hours'] = df['DIFF'] / 60
df['Speed_Miles'] = df['MILES*'] / df['Duration_hours']
df['Speed_Miles'].describe()
df.head()
#del df['Speed_KM']

## 3. Category & Purpose

In [ ]:
df['CATEGORY*'].value_counts()

**Most trips are in the business category**

In [ ]:
#Purpose
df['PURPOSE*'].value_counts()


**Most trips are for meetings**

In [ ]:
#Average distance traveled for each activity
df.groupby('PURPOSE*').mean()['MILES*'].sort_values(ascending = False)

Now lets try to answer some questions from this data.

Question1: How many miles was earned per category and purpose ?

Question2: What is percentage of business miles vs personal?

Question3: How much time was spend for drives per category and purpose? 


In [ ]:
#Question1: How many miles was earned per purpose ?
df.groupby('PURPOSE*').sum()['MILES*'].sort_values(ascending = False)

In [ ]:
#Question2: How many miles was earned per category  ?
Miles_per_cat=df.groupby('CATEGORY*').sum()['MILES*']

In [ ]:
Miles_per_cat*100/Miles_per_cat.sum()

In [ ]:
df.groupby('CATEGORY*').sum()['Duration_hours']

In [ ]:
df.groupby('PURPOSE*').sum()['Duration_hours']

## Summary of findings from analysis